In [7]:
import quandl
import time
import datetime
import pandas
import numpy as np

In [3]:
#enter stock symbol
input='aapl' 

In [2]:
quandl.ApiConfig.api_key = 'Vf8rgA91Rw4b2x_Lm3EJ'
currentdate = datetime.datetime.now().date()

In [23]:
data = quandl.get("WIKI/"+input+".11", start_date="2016-11-10", end_date=str(currentdate), collapse="none", transform="none", returns="pandas")
metadata = quandl.Dataset("WIKI/"+input).to_list()
name = metadata[-1].split(',')[0]

In [25]:
data.head()

,Adj. Close
Date,
2016-11-10,107.76
2016-11-11,108.43
2016-11-14,105.74
2016-11-15,107.11


In [26]:
data = quandl.get("WIKI/"+input, start_date="2016-11-10", end_date=str(currentdate), collapse="none", transform="none", returns="pandas")


In [27]:
data.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2016-11-10,111.09,111.090,105.8300,107.76,56512401,0,1,111.09,111.090,105.8300,107.76,56512401
2016-11-11,106.92,108.870,106.5500,108.43,33964663,0,1,106.92,108.870,106.5500,108.43,33964663
2016-11-14,107.32,107.809,104.0800,105.74,50480286,0,1,107.32,107.809,104.0800,105.74,50480286
2016-11-15,106.57,107.680,106.1593,107.11,31597144,0,1,106.57,107.680,106.1593,107.11,31597144


In [15]:
df.tail()

,Date,Adj. Close
27,2016-11-09,110.88
28,2016-11-10,107.76
29,2016-11-11,108.43
30,2016-11-14,105.74
31,2016-11-15,107.11


In [21]:
data = quandl.get_table('ZACKS/FC', paginate=True, ticker=['AAPL', 'MSFT'], per_end_date={'gte': '2015-11-11'})

In [22]:
data

,m_ticker,ticker,comp_name,comp_name_2,exchange,currency_code,per_end_date,per_type,per_code,per_fisc_year,...,stock_based_compsn_qd,cash_flow_oper_activity_qd,net_change_prop_plant_equip_qd,comm_stock_div_paid_qd,pref_stock_div_paid_qd,tot_comm_pref_stock_div_qd,wavg_shares_out,wavg_shares_out_diluted,eps_basic_net,eps_diluted_net
None,,,,,,,,,,,,,,,,,,,,,
0,AAPL,AAPL,APPLE INC,Apple Inc.,NSDQ,USD,2016-09-30,A,FR0,2016,...,NaN,NaN,NaN,NaN,NaN,NaN,5470.820,5500.281,8.35,8.31
1,AAPL,AAPL,APPLE INC,Apple Inc.,NSDQ,USD,2015-12-31,Q,QR-3,2016,...,1078,27463,-3612,-2969,NaN,-2969,5558.930,5594.127,3.30,3.28
2,AAPL,AAPL,APPLE INC,Apple Inc.,NSDQ,USD,2016-03-31,Q,QR-2,2016,...,1048,11601,-2336,-2902,NaN,-2902,5514.381,5540.886,1.91,1.90
3,AAPL,AAPL,APPLE INC,Apple Inc.,NSDQ,USD,2016-06-30,Q,QR-1,2016,...,1054,10634,-2809,-3187,NaN,-3187,5443.058,5472.781,1.43,1.42
4,AAPL,AAPL,APPLE INC,Apple Inc.,NSDQ,USD,2016-09-30,Q,QR0,2016,...,1030,16126,-3977,-3092,NaN,-3092,5470.820,5500.281,1.71,1.71
5,MSFT,MSFT,MICROSOFT CORP,Microsoft Corp.,NSDQ,USD,2016-06-30,A,FR0,2016,...,NaN,NaN,NaN,NaN,NaN,NaN,7925.000,8013.000,2.12,2.10
6,MSFT,MSFT,MICROSOFT CORP,Microsoft Corp.,NSDQ,USD,2015-12-31,Q,QR-3,2016,...,658,5316,-2024,-2868,NaN,-2868,7964.000,8028.000,0.63,0.62
7,MSFT,MSFT,MICROSOFT CORP,Microsoft Corp.,NSDQ,USD,2016-03-31,Q,QR-2,2016,...,672,10669,-2308,-2842,NaN,-2842,7895.000,7985.000,0.48,0.47
8,MSFT,MSFT,MICROSOFT CORP,Microsoft Corp.,NSDQ,USD,2016-06-30,Q,QR-1,2016,...,664,8464,-2655,-2821,NaN,-2821,7925.000,8013.000,0.40,0.40


In [ ]:
quandl.ApiConfig.api_key = 'Vf8rgA91Rw4b2x_Lm3EJ'
currentdate = datetime.datetime.now().date()
try:
    data = quandl.get("WIKI/"+input+".11", start_date="2016-10-01", end_date=str(currentdate), collapse="none", transform="none", returns="numpy")
    metadata = quandl.Dataset("WIKI/"+input).to_list()
    name = metadata[-1].split(',')[0]
    print time.ctime()
    print name
    today = data[-1][-1]
    last = data[-2][-1]
    print today, today - last, "{0:.2f}%".format((today - last) / last * 100)
except Exception, e:
    print str(e).split(')')[-1]